In [174]:
#!/usr/bin/env python
# coding: utf-8
import time
import bs4
import requests
import html

def escape_text(param):
    return param.replace('\xa0', ' ')

def data_cleanup(data):
    if '\n' in data:
        return list(filter(lambda x: x!='', data.split('\n')))
    else:
        return data

def get_data(name):
    name = name.replace(' ', '_')
    url = f"https://en.wikipedia.org/wiki/{name}"
    res = requests.get(url)
    html = res.text
    soup = bs4.BeautifulSoup(html, "html.parser")
    
    data = {'name' : name}
    for table in soup.find_all('table'):
        if(' '.join(table.get('class', [])) == 'infobox vcard'):
            data['logo'] = bs4.BeautifulSoup(str(table))("tr")[0]('td')[0].a.find_all('img')[0]['src']
            for row in bs4.BeautifulSoup(str(table))("tr"):
                if(len(row("th")) and len(row("td"))):
                    if(row("th")[0].text == 'Website'):
                        data[escape_text(row("th")[0].text)] = escape_text(row("td")[0].a['href'])
                    elif(row("th")[0].text in ['Formerly', 'Products', 'Divisions', 'Services', 'Traded as']):
                        listData = []
                        for txt in row("td")[0].find_all('li'):
                            listData.append(escape_text(txt.text))
                        data[escape_text(row("th")[0].text)] = listData
                    else:
                        data[escape_text(row("th")[0].text)] = data_cleanup(escape_text(row("td")[0].text))
    return data

company_details = []
#This list would be retrieved from the backend datastore
company_list = ['Walmart', 'Berkshire Hathaway', 'Apple Inc', 'Exxon Mobil', 'McKesson Corporation', 
          'UnitedHealth Group', 'CVS Health', 'General Motors', 'Ford']



In [175]:
column_mapping = {"Area served": "area_served", "Divisions": "divisions", "Formerly": "formerly", "Founded": "founded", "Founders": "founder", "Founder": "founder", "Headquarters": "headquarters", "ISIN": "ISIN", "Industry": "industry", "Key people": "key_people", "Net income": "net_income", "Number of employees": "employee_count", "Number of locations": "location_count","Predecessor":"predecessor",
                  "Operating income": "operating_income", "Owner": "owner", "Products": "product", "Revenue": "revenue", "Services": "services", "Subsidiaries": "subsidiares", "Total assets": "total_assets", "Total equity": "total_equity", "Traded as": "traded_as", "Type": "type", "Website": "website", "logo": "logo", "name": "company_name", "Production output":"production_output", "Owners": "owners"}


In [176]:
for company in company_list:
    company_details.append(get_data(company))

In [177]:
for detail in company_details:
    for key,value in column_mapping.items():
        if detail.get(key) is not None:
            detail[value] = detail.pop(key)    
    

In [178]:
len(company_details)

9

In [180]:
from pprint import pprint as print
print(company_details)


[{'ISIN': 'US9311421039',
  'area_served': 'Worldwide',
  'company_name': 'Walmart',
  'divisions': ['Walmart U.S.',
                'Walmart International',
                "Sam's Club",
                'Global eCommerce'],
  'employee_count': ['2.2 million, Worldwide (2018)[3]',
                     '1.5 million, U.S. (2017)[4]',
                     '700,000, International'],
  'formerly': ["Walton's (1950–1969)",
               'Wal-Mart, Inc. (1969–1970)',
               'Wal-Mart Stores, Inc. (1970–2018)'],
  'founded': 'June 13, 1945; 74 years ago (1945-06-13) (in Rogers, Arkansas)',
  'founder': 'Sam Walton',
  'headquarters': 'Bentonville, Arkansas, U.S.',
  'industry': 'Retail',
  'key_people': ['Greg Penner (Chairman)', 'Doug McMillon (President and CEO)'],
  'location_count': ' 11,389 stores worldwide (July 31, 2019)[1][2]',
  'logo': '//upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Walmart_logo.svg/220px-Walmart_logo.svg.png',
  'net_income': ' US$6.67 billion (2019)[3

#Writing to elasticsearch

In [181]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

ModuleNotFoundError: No module named 'elasticsearch'